## Red Bayesiana para identificar el sentimiento de las revisiones de peliculas.

En esta práctica vamos a crear una Red Bayesiana que ayude a identificar el sentimiento (opinión) de una pelicula que tiene un usuario a partir de la revisión que publicó en IMDB.

Utilizaremos una base de datos de IMDB [IMDB dataset](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) que contiene el texto de  50,000 revisiones de  [Internet Movie Database](https://www.imdb.com/). 

Todas las revisiones ya tienen una etiqueta *positive* o *negative*. Las revisiones están separadas en dos conjuntos: el conjunto de entrenamiento *train*, que utilizaremos para generar la Red Bayesiana y el conjunto de *test* que utilizaremos para comprobar la eficacia de la red a la hora de predecir la opinión del usuario. 

Este notebook utiliza  [nltk](https://www.nltk.org/)  para realizar operaciones con los textos. Si estas ejecutando este notebook en tu PC deberás instalar el paquete previamente.

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt

La siguiente celda carga el IMDB dataset:

In [2]:
imdb_data = pd.read_csv('IMDB Dataset.csv')
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Conjunto de revisiones

Cada ejemplo es un texto que representa la revisión de la pelicula y su *sentimiento* correspondiente. El texto no está procesado de ninguna forma.


In [3]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [4]:
#Dividir el dataset en un conjunto de entrenamiento para calcular las verosimilitudes y otro de test. 
#train dataset
train_examples=list(imdb_data.review[:40000])
train_labels=list(imdb_data.sentiment[:40000])
#test dataset
test_examples=list(imdb_data.review[40000:])
test_labels=list(imdb_data.sentiment[40000:])


In [5]:
print("Número de revisiones de entrenamiento: {}, de test: {}".format(len(train_examples), len(test_examples)))

Número de revisiones de entrenamiento: 40000, de test: 10000


Vemos las 5 primeras revisiones

In [6]:
train_examples[:5]

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

que tienen los siguientes *labels*:

In [7]:
train_labels[:5]

['positive', 'positive', 'positive', 'negative', 'positive']

Primero procesamos el texto de las revisiones para eliminar las puntuaciones y otros símbolos:

In [5]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews): 
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, str(line).lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, str(line)) for line in reviews]
    return reviews

train_reviews_clean = preprocess_reviews(train_examples)
test_reviews_clean = preprocess_reviews(test_examples)

Además, vamos a eliminar de los textos las palabras más comunes del inglés (que se denominan stopwords) ya que no aportan información sobre si el texto es una opinión positiva o negativa. En el siguiente ejemplo vemos como se eliminan las palabras más comunes:

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

ejemplo = "This is a sample sentence, showing off the stop words filtration."
stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(ejemplo)
filtered_sentence = [w for w in word_tokens if not w in stop_words]
print(word_tokens)
print(filtered_sentence)

['This', 'is', 'a', 'sample', 'sentence', ',', 'showing', 'off', 'the', 'stop', 'words', 'filtration', '.']
['This', 'sample', 'sentence', ',', 'showing', 'stop', 'words', 'filtration', '.']


[nltk_data] Downloading package punkt to /home/eduardo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eduardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Convertimos cada review en una lista con las palabras procesadas:

In [8]:
#lista de todas las palabras conjunto de train
train_reviews = []
for review in train_reviews_clean:
    tokens = nltk.word_tokenize(review)
    filtered_sentence = [w for w in tokens if not w in stop_words]
    train_reviews.append(filtered_sentence)

In [11]:
train_reviews[4]

['petter',
 'matteis',
 'love',
 'time',
 'money',
 'visually',
 'stunning',
 'film',
 'watch',
 'mr',
 'mattei',
 'offers',
 'us',
 'vivid',
 'portrait',
 'human',
 'relations',
 'movie',
 'seems',
 'telling',
 'us',
 'money',
 'power',
 'success',
 'people',
 'different',
 'situations',
 'encounter',
 'variation',
 'arthur',
 'schnitzlers',
 'play',
 'theme',
 'director',
 'transfers',
 'action',
 'present',
 'time',
 'new',
 'york',
 'different',
 'characters',
 'meet',
 'connect',
 'one',
 'connected',
 'one',
 'way',
 'another',
 'next',
 'person',
 'one',
 'seems',
 'know',
 'previous',
 'point',
 'contact',
 'stylishly',
 'film',
 'sophisticated',
 'luxurious',
 'look',
 'taken',
 'see',
 'people',
 'live',
 'world',
 'live',
 'habitat',
 'thing',
 'one',
 'gets',
 'souls',
 'picture',
 'different',
 'stages',
 'loneliness',
 'one',
 'inhabits',
 'big',
 'city',
 'exactly',
 'best',
 'place',
 'human',
 'relations',
 'find',
 'sincere',
 'fulfillment',
 'one',
 'discerns',
 'cas

## Red Bayesiana
La RB que vamos a utilizar para predecir si la revisión fué escrita por un usuario con una opinión positiva de la pelicula tiene una variable oculta, el sentimiento del usuario *positivo* o *negativo* hacia la pelicula y muchas variables de evidencia, que son las palabras que aparecen en su comentario. ![Red Bayesiana](sentimiento.jpg)

Por ejemplo, podemos pensar que si la opinión de es positiva entonces la palabra *good* tiene probabilidades altas de aparecer en el comentario. Es decir, que P(good|positivo) tendrá un valor alto. Para calcular el valor de las **verosimilitudes**, es decir de las probabilidades de que una palabra aparezca condicionado a que en el comentario sea positivo P(palabra|positivo)  vamos a utilizar el conjunto de entrenamiento (para el caso negativo igual). Para calcular la probabilidad de que una palabra aparezca condicionado a que sea un comentario positivo utilizaremos la siguiente división:

$P(palabra|positivo) = \frac{\# palabra-y-positivo}{\# palabra}$

Por ejemplo, para la palabra good dividiremos el número de veces que aparece la palabra good en un comentario positivo entre el número de veces totales que aparece la palabra good. 

Haz un diccionario 'conteo_palabra' en el que cuentes el número de veces que aparece una palabra en todas las revisiones de entrenamiento:

In [9]:
conteo_palabra = {}
for review in train_reviews:
    for word in review:
        if word in conteo_palabra:
            conteo_palabra[word] += 1
        else:
            conteo_palabra[word] = 1
        
print(conteo_palabra['good'])
      

23831


Ahora crea un diccionario en el que se almacenan las veces que aparece una palabra en un comentario positivo:

In [30]:
conteo_palabra_positivo = {}
for i in range (len(train_reviews)):
    if train_labels[i] == 'positive':
        for word in train_reviews[i]:
            if word in conteo_palabra_positivo:
                conteo_palabra_positivo[word] += 1
            else:
                conteo_palabra_positivo[word] = 1
conteo_palabra_negativo = {}
for i in range (len(train_reviews)):
    if train_labels[i] == 'negative':
        for word in train_reviews[i]:
            if word in conteo_palabra_negativo:
                conteo_palabra_negativo[word] += 1
            else:
                conteo_palabra_negativo[word] = 1
print(conteo_palabra_positivo['good'])

11999


Crea un función a la que le pasemos una palabra y devuelva la probabilidad:

$P(palabra|positivo) = \frac{\# positivo-y-palabra}{\# palabra}$

Si ocurre que la palabra no ha aparecido nunca en un comentario positivo la función debe devolver un valor pequeño, por ejemplo 0.01.

In [35]:
def prob_pos_cond(word):
    if word in conteo_palabra_positivo:
        return conteo_palabra_positivo[word] / conteo_palabra[word]
    return 0.01
def prob_neg_cond(word):
    if word in conteo_palabra_negativo:
        return conteo_palabra_negativo[word]/ conteo_palabra[word]
    return 0.01

print(prob_pos_cond('good'))

0.5035038395367378


In [36]:
ejemplo = ['good', 'nice', 'awesome', 'terrific', 'horrible', 'worst', 'boring' ] 
for p in ejemplo:
    print(prob_pos_cond(p))

0.5035038395367378
0.5599214145383105
0.7178082191780822
0.8100303951367781
0.13615502294747578
0.08684396328547894
0.18162911611785096


La distribución de la Red Bayesiana la podemos expresar como:

$P(sentimiento, palabra_1, palabra_2,...) = P(sentimiento) \cdot \prod_i P(palabra_i|sentimiento)$

Por tanto, para calcular la probabilidad de que una revisión sea una opinión positiva hay que calcular el producto de $P(positivo) \cdot \prod_i P(palabra_i|positivo)$  para la revisión y ver si es mayor que la probabilidad de que la frase sea negativa $P(negativo) \cdot \prod_i P(palabra_i|negativo)$. 
Por la forma en la que calculamos P(palabra_i|positivo), tenemos que  $P(palabra_i|negativo) = 1 - P(palabra_i|positivo)$ (aunque ya vimos que esto no es cierto en general).

Ten en cuenta que tienes que calcular  P(positive) y P(negative) para el conjunto de entrenamiento.

### Importante:
Como las probabilidades son valores entre 0 y 1 y las revisiones pueden tener muchas palabras, en lugar de realizar la multiplicación de las probabilidades, se calcula **la suma de los logaritmos de las probabilidades** para evitar llegar al cero float.

Haz dos funciones diferentes, una que devuelva la probabilidad de que una review sea un comentario positivo y otra que devuelva la probabilidad de ser un comentario negativo (la probabilidad debe ser la suma de los logaritmos).

In [37]:
import math
def prob_frase_pos(frase):
    prob = 0
    for word in frase:
        prob += math.log(prob_pos_cond(word))
    return prob

def prob_frase_neg(frase):
    prob = 0
    for word in frase:
        prob += math.log(prob_neg_cond(word))
    return prob

Utilizando el propio conjunto de entrenamiento calcula el porcentaje de comentarios que la red bayesiana predice correctamente el sentimiento del comentario:

In [39]:
# Comprobar el acierto sobre el conjunto de train
aciertos=0
for i in range (len(train_reviews)):
    if prob_frase_pos(train_reviews[i]) > prob_frase_neg(train_reviews[i]) and train_labels[i] == 'positive':
        aciertos += 1
    if prob_frase_pos(train_reviews[i]) < prob_frase_neg(train_reviews[i]) and train_labels[i] == 'negative':
        aciertos += 1
print('Porcentaje de acierto:', aciertos/len(train_reviews))

Porcentaje de acierto: 0.94565


Modifica las funciones para calcular la probabilidad de que una frase sea positiva o negativa para que el parámetro de entrada sea una revisión original del conjunto de test. La función debe procesar la frase, eliminar caracteres, eliminar las stop words y convertir la review en una lista de palabras. Además debes tener en cuenta que puede haber palabras en las revisiones de test que no han aparecido en el conjunto de entrenamiento. Para estos casos, no se suma ninguna probabilidad.

Calcula el porcentaje de acierto para el conjunto de revisiones de test.

In [43]:
test_reviews_clean = preprocess_reviews(test_examples)
stop_words = set(stopwords.words('english'))
test_reviews = []
for review in test_reviews_clean:
    tokens = nltk.word_tokenize(review)
    filtered_sentence = [w for w in tokens if not w in stop_words]
    test_reviews.append(filtered_sentence)
conteo_palabra = {}
for review in test_reviews:
    for word in review:
        if word in conteo_palabra:
            conteo_palabra[word] += 1
        else:
            conteo_palabra[word] = 1
conteo_palabra_positivo = {}
for i in range (len(test_reviews)):
    if test_labels[i] == 'positive':
        for word in test_reviews[i]:
            if word in conteo_palabra_positivo:
                conteo_palabra_positivo[word] += 1
            else:
                conteo_palabra_positivo[word] = 1
conteo_palabra_negativo = {}
for i in range (len(test_reviews)):
    if test_labels[i] == 'negative':
        for word in test_reviews[i]:
            if word in conteo_palabra_negativo:
                conteo_palabra_negativo[word] += 1
            else:
                conteo_palabra_negativo[word] = 1

def probabilidad (test_examples):
    test_reviews_clean = preprocess_reviews(test_examples)
    stop_words = set(stopwords.words('english'))
    test_reviews = []
    for review in test_reviews_clean:
        tokens = nltk.word_tokenize(review)
        filtered_sentence = [w for w in tokens if not w in stop_words]
        test_reviews.append(filtered_sentence)
    aciertos=0
    for i in range (len(test_reviews)):
        if prob_frase_pos(test_reviews[i]) > prob_frase_neg(test_reviews[i]) and test_labels[i] == 'positive':
            aciertos += 1
        if prob_frase_pos(test_reviews[i]) < prob_frase_neg(test_reviews[i]) and test_labels[i] == 'negative':
            aciertos += 1
    return aciertos / len(test_reviews)

test_examples=list(imdb_data.review[40000:])
test_labels=list(imdb_data.sentiment[40000:])
print(probabilidad(test_examples))
    

0.9772
